In [1]:
import json
import pandas as pd
import codecs
from pandas.io.json import json_normalize as jsy
#!pip install gensim
from gensim.models import Word2Vec
from nltk.corpus import brown # , #movie_review

C:\Users\gokul\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
dev_set = pd.read_json('C:/Users/gokul/Documents/Projects/selqa-evaluater/SelQA-ass-dev.json', lines=True)
train_set = pd.read_json('C:/Users/gokul/Documents/Projects/selqa-evaluater/SelQA-ass-train.json', lines = True)
test_set = pd.read_json('C:/Users/gokul/Documents/Projects/selqa-evaluater/SelQA-ass-test.json', lines = True)

In [3]:
train_set.head()

,article,candidates,is_paraphrase,question,section,sentences,type
0,Table tennis,[4],False,Does a 40mm table tennis ball spin more or les...,Ball,[The international rules specify that the game...,SPORT
1,Blow Me (One Last Kiss),[10],False,What does Pink wear in the video?,Music video,"[The music video for ""Blow Me (One Last Kiss)""...",MUSIC
2,Chile,[16],True,In what year was the Chilean National Museum o...,Tourism,[Tourism in Chile has experienced sustained gr...,COUNTRY
3,Battle of Hastings,[6],False,"How many times was the who national ""fyrd"" cal...",English army and Harold's preparations,[The English army was organised along regional...,HISTORICAL EVENTS
4,The Mindy Project,[6],False,Who did Chris Messina play on The Mindy Project?,Production and development,"[The series was initially commissioned by NBC,...",TV


In [54]:
test_set[test_set['article']=='Table tennis']
x = test_set.loc[1422]['sentences']
train_set[(train_set['article']=='Table tennis')&(train_set['section']=='Ball')]

temp = (train_set.loc[0]['sentences'])
j=0
for i in range(len((train_set.loc[0]['sentences']))):
    print(j)
    print(temp[i])
    j+=1

0
The international rules specify that the game is played with a light 2.7 gram, 40 mm diameter ball.
1
The rules say that the ball shall bounce up 24–26 cm when dropped from a height of 30.5 cm onto a standard steel block thereby having a coefficient of restitution of 0.89 to 0.92.
2
The 40 mm ball was introduced after the 2000 Olympic Games.
3
However, this created some controversy as the Chinese National Team argued that this was merely to give non-Chinese players a better chance of winning since the new type of balls has a slower speed, while at that time most Chinese players were playing with fast attack and smashes.
4
A 40 mm table tennis ball is slower and spins less than the original 38 mm (1.5 inch) one.
5
The ball is made of a high-bouncing air-filled celluloid or similar plastics material, colored white or orange, with a matte finish.
6
The choice of ball color is made according to the table color and its surroundings.
7
For example, a white ball is easier to see on a green 

In [81]:
train = pd.DataFrame(train_set['question'])
train['answers'] = "NA"
train.shape

temp1 = len(train_set)
for i in range(temp1):
    #print(type(train_set.loc[i]['question']))
    temp = len(train_set.loc[i]['candidates'])
    if temp==1:
        anskey = train_set.loc[i]['candidates'][0]
        #print(type(train_set.loc[i]['sentences'][anskey]))
        train.loc[i]['answers'] = train_set.loc[i]['sentences'][anskey]
    else:
        pass

In [85]:
writetrain = train[train['answers']!="NA"]
writetrain.to_csv("./train.csv")

In [93]:
from emo_utils import *

read_csv("train.csv")

ValueError: invalid literal for int() with base 10: 'A 40\xa0mm table tennis ball is slower and spins less than the original 38\xa0mm (1.5\xa0inch) one.'

,question,answers


In [3]:

b = Word2Vec(brown.sents())
b.most_similar('money', topn=5)

C:\Users\gokul\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


[('care', 0.9206393361091614),
 ('job', 0.9165905714035034),
 ('trouble', 0.9065046310424805),
 ('work', 0.8878711462020874),
 ('getting', 0.8855317234992981)]

In [4]:
type(brown.sents())

nltk.corpus.reader.util.ConcatenatedCorpusView

In [7]:
x = list(brown.sents())

In [8]:
len(x)

57340

In [9]:
len(x[10])

14

In [10]:
x[10]

['It',
 'urged',
 'that',
 'the',
 'city',
 '``',
 'take',
 'steps',
 'to',
 'remedy',
 "''",
 'this',
 'problem',
 '.']

In [14]:
b.build_vocab_from_freq(word_freq=3)

TypeError: object of type 'int' has no len()